In [ ]:
! pip install -q claude langchain psycopg2

In [ ]:
from langchain.utilities import SQLDatabase

useranme = ''
password = ''
host = 'localhost'
port =''
database = 'postgres'

pg_uri = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"

db = SQLDatabase.from_uri(pg_uri)

In [ ]:
def get_scheme(_):
    return db.get_table_info()

def run_query(query):
    return db.run(query)

In [ ]:
# 
from langchain.prompts import ChatPromptTemplate

template_query = """
DB데이터 정보 schema를 통해, \
사용자 질문 question이 주어졌을 때 PostgreSQL Query를 생성하라:
{schema}

Question: {question}
SQL Query:"""

prompt = ChatPromptTemplate.from_template(template_query)


In [ ]:
from langchain.chat_models import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

model = ChatAnthropic(
    model_name="claude-haiku-4-5-20251001",
    temperature=0.2,
    max_tokens=1024,
)
sql_response = (
    RunnablePassthrough.assign(schema=get_schema) 
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [ ]:
sql_response.invoke({"question": "서울에 사는 20대 남성 100명 회원번호 출력해줘"})

In [ ]:
template_response = """
DB테이블 정보, 사용자 질문, SQL Query, SQL Query 결과가 있을 때\
자연어로 답변 생성해줘:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""

prompt_response = ChatPromptTemplate.from_template(template_response)

In [ ]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema, 
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | model
    | StrOutputParser()
)